In [41]:
 pip install MetaTrader5

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
# import trinket 

In [2]:
# подключимся к MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    quit()

In [7]:
#Получим тики EURRUB и USDRUB со вчерашнего дня по сегодняшний день
eurrub_ticks = mt5.copy_ticks_range("EURUSD", 
                                     datetime(2020,1,9,13), 
                                     datetime(2020,1,10,13), 
                                     mt5.COPY_TICKS_ALL)

usdrub_ticks = mt5.copy_ticks_range("GBPUSD", 
                                     datetime(2020,1,9,13), 
                                     datetime(2020,1,10,13),
                                     mt5.COPY_TICKS_ALL)

In [8]:
eurrub_ticks

array([(1578556800, 1.11146, 1.11159, 1.11147, 1, 1578556800220,   2, 1.),
       (1578556800, 1.11146, 1.11159, 1.11146, 1, 1578556800275, 280, 1.),
       (1578556800, 1.11148, 1.11161, 1.11146, 1, 1578556800597,   6, 1.),
       ...,
       (1578643197, 1.11064, 1.11077, 1.11064, 1, 1578643197699, 280, 1.),
       (1578643198, 1.11065, 1.11077, 1.11064, 1, 1578643198362,   2, 1.),
       (1578643198, 1.11065, 1.11077, 1.11065, 1, 1578643198362, 280, 1.)],
      dtype=[('time', '<i8'), ('bid', '<f8'), ('ask', '<f8'), ('last', '<f8'), ('volume', '<u8'), ('time_msc', '<i8'), ('flags', '<u4'), ('volume_real', '<f8')])

In [9]:
# подготовка запроса
symbol_dol = "GBPUSD"
symbol_euro = "EURUSD"
symbol_dol_info =mt5.symbol_info(symbol_dol)
symbol_euro_info =mt5.symbol_info(symbol_euro)
# lot = 1.0
lot = 0.01 
point_dol = mt5.symbol_info(symbol_dol).point 
price_dol = mt5.symbol_info_tick(symbol_dol).ask
price_euro = mt5.symbol_info_tick(symbol_euro).ask
point_euro = mt5.symbol_info(symbol_euro).point
# deviation = отклонение
deviation = 20 
# запросы

# покупка доллара
request = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_dol, 
"volume": lot, 
"type": mt5.ORDER_TYPE_BUY, 
"price": price_dol, 
"stoplimit": price_dol*1.25,
#"sl": price_dol - 100 * point_dol, 
#"tp": price_dol + 100 * point_dol, 
"deviation": deviation, 
"magic": 123456, 
"comment": "купили доллар", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}

# продажа доллара
request_2 = {
"action": mt5.TRADE_ACTION_DEAL, # Совершить сделку прямо сейчас
"symbol": symbol_dol, # Сделка USDRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL, # Покупаем или продаём
#"price": price, # Для .TRADE_ACTION_DEAL не требуется
#"sl": price - 100 * point,
#"tp": price + 100 * point,
"deviation": deviation, # Максимальное отклонение от заданной цены
"magic": 123456,
"comment": "продали доллар",
"type_time": mt5.ORDER_TIME_GTC,
# "type_filling": mt5.ORDER_FILLING_IOC,
"type_filling": mt5.ORDER_FILLING_FOK,   #не будет продажи если нет необходимого обьема 
    
                                            #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
#  struct MqlTradeRequest
#   {
#    ENUM_TRADE_REQUEST_ACTIONS    action;           // Тип выполняемого действия
#    ulong                         magic;            // Штамп эксперта (идентификатор magic number)
#    ulong                         order;            // Тикет ордера
#    string                        symbol;           // Имя торгового инструмента
#    double                        volume;           // Запрашиваемый объем сделки в лотах
#    double                        price;            // Цена 
#    double                        stoplimit;        // Уровень StopLimit ордера
#    double                        sl;               // Уровень Stop Loss ордера
#    double                        tp;               // Уровень Take Profit ордера
#    ulong                         deviation;        // Максимально приемлемое отклонение от запрашиваемой цены
#    ENUM_ORDER_TYPE               type;             // Тип ордера
#    ENUM_ORDER_TYPE_FILLING       type_filling;     // Тип ордера по исполнению
#    ENUM_ORDER_TYPE_TIME          type_time;        // Тип ордера по времени действия
#    datetime                      expiration;       // Срок истечения ордера (для ордеров типа ORDER_TIME_SPECIFIED)
#    string                        comment;          // Комментарий к ордеру
#    ulong                         position;         // Тикет позиции
#    ulong                         position_by;      // Тикет встречной позиции
#   };
   
request2 = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro, # Сделка EURRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL,
"price": price_euro,
#sl": 100000,
#tp": 100000,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продажа евро",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,   #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
request2_2= {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": "EURRUB", # Сделка EURRUB
#"volume": 1.0, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_BUY,
#"price": price,
#sl": 100000,
#tp": 100000,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "python script open",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,      #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
# request-ПОКУПКА ДОЛЛАРА
# request_2-ПРОДАЖА ДОЛЛАРА
# request2-ПРОДАЖА ЕВРО
# request2_2-ПОКУПКА ЕВРО


In [ ]:
test_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": OnLot(),
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price - SL * point,
            "tp": price + TP * point,
            "deviation": deviation,
            "magic": magic,
            "comment": "python script open",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK,
}

In [26]:
# Завершим подключение - // завершать подключение будем в самом конце а если программа 
# будет использоваться все время то и откл не будем


In [10]:
#Переведем имеющиеся у нас данные в DataFrame
eurrub_ticks_frame = pd.DataFrame(eurrub_ticks)
usdrub_ticks_frame = pd.DataFrame(usdrub_ticks)
#Время в MetaTrader 5 хранится в UTC таймзоне, поэтому переведем в привычный нам вид
eurrub_ticks_frame['time']=pd.to_datetime(eurrub_ticks_frame['time'], unit='s')
usdrub_ticks_frame['time']=pd.to_datetime(usdrub_ticks_frame['time'], unit='s')

#Удалим ненужные столбцы
eurrub_ticks_frame.drop('ask', axis = 1, inplace = True)
eurrub_ticks_frame.drop('flags', axis = 1, inplace = True)
eurrub_ticks_frame.drop('volume_real', axis = 1, inplace = True)
eurrub_ticks_frame.drop('last', axis = 1, inplace = True)
eurrub_ticks_frame.drop('volume', axis = 1, inplace = True)

usdrub_ticks_frame.drop('bid', axis = 1, inplace = True)
usdrub_ticks_frame.drop('flags', axis = 1, inplace = True)
usdrub_ticks_frame.drop('volume_real', axis = 1, inplace = True)
usdrub_ticks_frame.drop('last', axis = 1, inplace = True)
usdrub_ticks_frame.drop('volume', axis = 1, inplace = True)
#eurrub_ticks_frame['time_msc']=pd.to_datetime(eurrub_ticks_frame['time_msc'], unit='ms')
#usdrub_ticks_frame['time_msc']=pd.to_datetime(usdrub_ticks_frame['time_msc'], unit='ms')

In [11]:
position_euro = 0
position_usd = 0
old_euro = eurrub_ticks_frame['bid'][0]
old_usd = usdrub_ticks_frame['ask'][0]
current_usd = old_usd
current_euro = old_euro

spread = []

while(position_euro <= len(eurrub_ticks_frame['time'])):
    
    if(eurrub_ticks_frame['time_msc'][position_euro] >= usdrub_ticks_frame['time_msc'][position_usd]):
        
        current_usd = usdrub_ticks_frame['ask'][position_usd]
        spread.append(current_euro - current_usd)
        old_usd = usdrub_ticks_frame['ask'][position_usd]
        if position_usd >= len(usdrub_ticks_frame['time']) - 1:
            break
        position_usd += 1
    else:
        
        current_euro = eurrub_ticks_frame['bid'][position_euro]
        spread.append(current_euro - old_usd)
        if position_euro >= len(eurrub_ticks_frame['time']) - 1:
            break
        position_euro += 1
    
        

In [12]:
len(spread)

225212

In [13]:
#максимальное и минимальное значение
max_spread = max(spread)
min_spread = min(spread)

In [14]:
#Найдём медиану
spread = np.array(spread)
type(spread)
median = np.median(spread)
median

-0.19523000000000001

In [15]:
max_spread

-0.19106

In [16]:
min_spread

-0.20001999999999986

In [17]:
Open = median + (max_spread - median) * 0.8
print(Open)
Close = median + (min_spread - median) * 0.8
print(Close)


-0.191894
-0.1990619999999999


In [18]:
def request_open_pos(): 
    result =mt5.order_send(request) 
    result2 = mt5.order_send(request2) 
    print(result)
    print(result2)

In [19]:
def request_close_pos():
    result = mt5.order_send(request_2) 
    result2 = mt5.order_send(request2_2) 
    print(result)
    print(result2)

In [20]:
# while(True):
#     last_tick_usdrub = mt5.symbol_info_tick("EURRUB")
#     last_tick_GPBUSD = mt5.symbol_info_tick("USDRUB")
#     last_spread = last_tick_GPBUSD - last_tick_usdrub
#     if last_spread > Open:
#         open_pos() # Написать Функции
#     elif last_spread < Close*1.01:
#         close_pos() # Написать Функции
while(True):
    last_tick_eurrub = mt5.symbol_info_tick("EURRUB")
    last_tick_usd = mt5.symbol_info_tick("USDRUB")
    last_spread = last_tick_eurrub.bid  - last_tick_usd.ask
    result = mt5.order_send(request)
    print(result)
    #request_open_pos()
    time.sleep(1)
    #request_close_pos()
    time.sleep(1)
    #Комиссию учитывать

AttributeError: 'NoneType' object has no attribute 'bid'

In [6]:
mt5.shutdown()

True

In [59]:
last_tick_usdrub = mt5.symbol_info_tick("USDRUB") 

In [60]:
last_tick_usdrub

In [ ]:
###  Далее черновой код не использующийся в основной программе

In [ ]:

lasttick=mt5.symbol_info_tick("GBPUSD")


In [8]:
define EXPERT_MAGIC 123456   // MagicNumber эксперта
def OnStart():
   MqlTradeRequest request={0};
   MqlTradeResult  result={0};

   request.action   =TRADE_ACTION_DEAL;                     # тип торговой операции
   request.symbol   ="GBPUSD";                              # символ
   request.volume   =0.1;                                   # объем в 0.1 лот
   request.type     =ORDER_TYPE_BUY;                        # тип ордера
   request.price    =SymbolInfoDouble(Symbol(),SYMBOL_ASK); # цена для открытия
   request.deviation=5;                                     # допустимое отклонение от цены
   request.magic    =EXPERT_MAGIC;                         # MagicNumber ордера

   if(!OrderSend(request,result))
      PrintFormat("OrderSend error %d",GetLastError());     # если отправить запрос не удалось, вывести код ошибки
   PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

SyntaxError: invalid syntax (<ipython-input-8-7a7daa745a3e>, line 1)

In [ ]:
PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

In [ ]:
def user_login():
    try:
    login(
    login,                    // номер счета
    password="PASSWORD",      // пароль
    server="SERVER",          // имя сервера, как оно задано в терминале
    timeout=TIMEOUT           // таймаут
    )
    except  TimeoutErrore

# account=25115284
# authorized=mt5.login(account, password="gqrtz0lbdm")
# user_login()

def errors():
     print("initialize() failed, error code =",mt5.last_error())
#   

In [ ]:
Class Money:
    name = "RUB"
    price
# 
Class Dollar(Money):
    price
# 

Class Euro(Money):
    price
    def __init__(self,price):
        self.price = price
    def __del__():
        print(seld.price,"удален")
# 
euro = Euro()
